# Input/Outpt (IO)
Once you've processed a text or corpus with medspaCy, a next step is often to save and analyze the information you've extracted. `medpsacy.io` contains utilities for reading documents, converting processed docs into structured data, and writing your results to disk or to a database.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.insert(0, "..")

In [4]:
import medspacy

In [5]:
import sqlite3

#If you haven't already, install this pre-trained [med7](https://github.com/kormilitzin/med7) model and `spacy-transformers`:

In [6]:
# !pip install https://github.com/abchapman93/spacy_models/raw/master/releases/en_info_3700_i2b2_2012-0.1.0/dist/en_info_3700_i2b2_2012-0.1.0.tar.gz
# !pip install spacy-transformers

In [7]:
enable = ['pyrush',
 'tagger',
 'parser',
 'ner',
 'target_matcher',
 'context',
 'sectionizer']
nlp = medspacy.load("en_core_med7_trf", enable=enable)

OSError: [E050] Can't find model 'en_core_med7_trf'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
nlp.pipe_names

As an example, we'll use a very simple sqlite database containing two sample documents.

In [ ]:
import sqlite3 as sqlite

In [ ]:
from helpers import create_medspacy_demo_db

In [ ]:
create_medspacy_demo_db(drop_existing=True)

In [ ]:
conn = sqlite.connect("medspacy_demo.db")

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute("SELECT text FROM texts;")

In [ ]:
text = cursor.fetchone()[0]

In [ ]:
print(text[:600])

In [ ]:
conn.close()

In [ ]:
doc = nlp(text)

# I. DocConsumer
The `DocConsumer` class takes the attributes extracted by medspaCy and converts them into structured data. There are four different data types that the `DocConsumer` will extract:
- **"ent"**: Extract information about the spans in `doc.ents`. Each row will represent a single entity and can include either native spaCy attributes (ie., `ent.label_`) or custom attrributes (ie., `ent._.is_negated`)
- **"section"**: Each row will represent a section of the document and includes attributes such as the section text and category
- **"context"**: This represents the entity-modifier pairs extracted by ConText
- **"doc"**: A single row for the entire doc. By default this will only include `doc.text`, but you can add other underscore attributes

Let's create a `DocConsumer` with all four of these data types. We'll use the default attributes for now but will show how to customize them later.

In [ ]:
from medspacy.io import DocConsumer

In [ ]:
doc_consumer = DocConsumer(nlp, dtypes=("ent", "context", "section", "doc"))

`dtype_attrs` maps the data types to the corresponding attributes/columns

In [ ]:
doc_consumer.dtype_attrs

Now when we process our doc, we can get the relevant information from the `doc._.data` attribute. This is a nested dictionary where the outermost keys are the data types, and for each data type is an ordered dictionary mapping an attribute to the column of values for each ent.

In [ ]:
doc = doc_consumer(doc)

In [ ]:
doc._.data

If you have pandas installed, you can also now convert your doc to a DataFrame for each of the four levels:

In [ ]:
# !pip install pandas

In [ ]:
doc._.to_dataframe("ent").head(10)

Now let's go through each of the 4 levels individually. You can specify a single level of data by either passing in the dtype to `doc._.get_data(dtype)` or accessing individual attributes.

## Ents data

In [ ]:
ent_data = doc._.ent_data

In [ ]:
ent_data.keys()

You can also access the data row-wise rather then column-wise:

In [ ]:
doc._.get_data("ent", as_rows=True)[:2]

In [ ]:
ents_df = doc._.to_dataframe("ent")

In [ ]:
ents_df.head()

In [ ]:
ents_df[ents_df["is_negated"] == True]

## Section data

In [ ]:
section_data = doc._.section_data

In [ ]:
section_data.keys()

In [ ]:
doc._.get_data("section", as_rows=True)[0]

In [ ]:
section_df = doc._.to_dataframe("section")

In [ ]:
section_df.head()

## Context data

In [ ]:
context_data = doc._.context_data

In [ ]:
context_data.keys()

In [ ]:
doc._.get_data("context", as_rows=True)[:2]

In [ ]:
context_df = doc._.to_dataframe("context")

In [ ]:
context_df.head()

## Doc

In [ ]:
doc_data = doc._.doc_data

In [ ]:
doc_data.keys()

In [ ]:
doc_df = doc._.to_dataframe("doc")

In [ ]:
doc_df

## Customizing attributes
You can customize the values in `dtype_attrs` to modify what attributes are stored. "doc" and "ent" dtypes can take additional attributes which aren't included in the default, but "section" and "context" attributes can only take subsets of the defaults.

You can see the default values by calling the class method below:

In [ ]:
DocConsumer.get_default_attrs()

Let's create a second model here and add a second `DocConsumer` with customized attributes. We'll also add a new custom attribute for `Doc`, `report_id`, and include it.

In [ ]:
from spacy.tokens import Doc
Doc.set_extension("report_title", default="")

In [ ]:
doc2 = nlp("Previously on antibiotics.")

In [ ]:
doc2._.report_title = "example_document"

In [ ]:
doc_consumer2 = DocConsumer(nlp, dtypes=("ent", "doc"), 
                            dtype_attrs={
                                "ent": [
                                    "text",
                                    "label_",
                                    "is_negated",
                                    "section_category",
                                    
                                ],
                                "doc":
                                ["text", "report_title"]
                                
                            }
)

In [ ]:
doc_consumer2(doc2)

In [ ]:
doc2._.to_dataframe("ent")

In [ ]:
doc2._.to_dataframe("doc")

# Writer and Reader
The reader and writer classes are utilities for I/O. Here we'll show how to use it to connect to a sqlite database, read in text, and write them back to a new table.

In [ ]:
from medspacy.io import DbWriter, DbReader, DbConnect

## DbConn
DbConnect is a wrapper for either a pyodbc or sqlite3 connection. It can then be passed into the DbReader and DbWriter classes to retrieve/store document data.

You can pass in either information for a pyodbc connection string or directly pass in a sqlite or pyodbc connection object. Here, we'll pass in a connection to our sqlite database.

In [ ]:
sq3_conn = sqlite3.connect("./medspacy_demo.db")

In [ ]:
cursor = sq3_conn.cursor()

In [ ]:
conn = DbConnect(conn=sq3_conn)

## DbReader
DbReader is a utility fo reading docs from a database. It takes two arguments:
- **db_conn**: A `DbConnect` object
- **read_query**: The SQL query to read documents

Calling `reader.read()` will then return the results of the specified query.

In [ ]:
read_query = """
SELECT text
FROM texts
"""

In [ ]:
reader = DbReader(conn, read_query)

In [ ]:
rslts = reader.read()

In [ ]:
texts = [r[0] for r in rslts]

In [ ]:
print(texts[0][:100])

In [ ]:
print(texts[1])

#### Now we will process these docs and prepare to write them back:

In [ ]:
nlp.add_pipe("medspacy_doc_consumer");

In [ ]:
docs = list(nlp.pipe(texts))

## DbWriter
DbWriter is a utility class for writing structured data back to a database. Here we'll use it to store out processed doc to a new table called `ents`.

DbWriter takes the following arguments:

- **destination_table**: The name of the table to insert values into
- **cols**: Column names of the destination table
- **col_types**: SQL data types of the columns
- **doc_dtype**: DocConsumer data type, default 'ent'
- **create_table**: Whether or not to create the table if it does not exist, default False
- **drop_existing**: Whether to drop the table if it does exist, default False,
- **write_batch_size**: Default 100


Our column names will be the "ents" attributes in our consumer:

In [ ]:
doc_consumer.dtype_attrs["ent"]

Now we'll define the SQL datatypes for each column:

In [ ]:
col_types = [
    "varchar(1000)",
    "int",
    "int",
    "varchar(100)",
    "int",
    "int",
    "int",
    "int",
    "int",
    "varchar(100)",
    "varchar(100)"
]

In [ ]:
for (name, typ) in zip(doc_consumer.dtype_attrs["ent"], col_types):
    print(name, typ)

No we'll instantiate our writer and write the doc's entities to a new table:

In [ ]:
writer = DbWriter(conn, "ents", cols=doc_consumer.dtype_attrs["ent"], 
                  col_types=col_types,
                  doc_dtype="ent",
                 create_table=True, drop_existing=True)

In [ ]:
for doc in docs:
    writer.write(doc)

Now we can query our table to retrieve the data we wrote:

In [ ]:
query = """
SELECT *
FROM ents
LIMIT 2;
"""

In [ ]:
cursor.execute(query)

In [ ]:
cursor.fetchall()

In [ ]:
query = """
SELECT label_, COUNT(1)
FROM ents
GROUP BY label_
"""

In [ ]:
cursor.execute(query)

In [ ]:
cursor.fetchall()

## Pipeline
Once we have our reader and writer objects, we can pass them in to a `Pipeline`, which will read texts in batches, process them with NLP, and write them back to the database as specified by the writer.

`Pipeline` takes the following arguments:
- **reader: A DbReader object**
- **writer: A Dbwriter object**
- **nlp: A spaCy model**
- **dtype: The DocConsumer data type to write to a database. Default "ent**

In order for the pipeline to work, the first column of both the source and destination tables must some sort of document identifier (ie., "text_id"). 

In [ ]:
from medspacy.io import Pipeline

In [ ]:
sq3_conn_read = sqlite3.connect("./medspacy_demo.db")
conn = DbConnect(conn=sq3_conn_read)

In [ ]:
reader = DbReader(conn, read_query="SELECT text_id, text FROM texts")
writer = DbWriter(conn, "ents", 
                  # Add the text_id to the column names and types
                  cols=["text_id"]+doc_consumer.dtype_attrs["ent"], 
                  col_types=["int"] + col_types,
                  doc_dtype="ent",
                 create_table=True, drop_existing=True)

In [ ]:
pipeline = Pipeline(nlp, reader, writer, nlp, dtype="ent")

In [ ]:
pipeline.process()

In [ ]:
conn = sqlite3.connect("./medspacy_demo.db")
cursor = conn.cursor()

In [ ]:
cursor.execute("""
SELECT text_id, label_, COUNT(1)
FROM ents
GROUP BY text_id, label_
""")

In [ ]:
cursor.fetchall()